In [2]:
import cobra
import time
import numpy as np
import more_itertools as mi
import multiprocessing as mp
import random

In [3]:
from cobra.flux_analysis import (
    single_gene_deletion, single_reaction_deletion, double_gene_deletion,
    double_reaction_deletion)

In [4]:
from cobra import Model, Reaction, Metabolite

In [5]:
start=time.time()

In [6]:
cobra.Configuration().solver='cplex'

In [7]:
%%time
model=cobra.io.read_sbml_model("./models/iJO1366.xml")

CPU times: user 3.49 s, sys: 72 ms, total: 3.56 s
Wall time: 3.56 s


In [8]:
model2=model.copy()

In [9]:
model2

Name,iJO1366
Memory address,7efd3dc62630
Number of metabolites,1805
Number of reactions,2583
Number of genes,1367
Number of groups,0
Objective expression,1.0*BIOMASS_Ec_iJO1366_core_53p95M - 1.0*BIOMASS_Ec_iJO1366_core_53p95M_reverse_5c8b1
Compartments,"cytosol, extracellular space, periplasm"


In [10]:
for j in range(len(model2.reactions)):
    r=model2.reactions[j]
    if r.id=="BIOMASS_Ec_iJO1366_core_53p95M":
        biomass=j
biomass

18

In [11]:
model2.objective=model2.reactions[biomass]
model2.pobjective_direction="max"
maxBio=model2.slim_optimize()

In [12]:
o2,etoh = 184,340

In [13]:
def protect(inicio,fin):
    prot=set()
    for i in range(inicio,fin):
        if i<len(model2.reactions):
            r=list(model2.reactions)[i]
            with model2:
                r.knock_out()
                model2.objective=model2.reactions[biomass]
                model2.objective_direction="max"
                sol1=model2.slim_optimize(error_value=-1)
                if sol1<0.1:
                    prot.add(i)
                else:
                    model2.objective=model2.reactions[etoh]
                    model2.objective_direction="max"
                    sol2=model2.slim_optimize(error_value=-1)
                    if sol2<=0.1:
                        prot.add(i)
    return(prot)

In [14]:
%%time
protected=set()
cpus=64

k=0
argus=[]
n=len(model2.reactions)
num=int(np.floor(n/cpus))
while k*num<n:
    argus.append((k*num,(k+1)*num))
    k+=1

with mp.Pool(processes=cpus) as mp_pool:
    results=mp_pool.starmap(protect,argus)

for result in results:
    protected.update(result)

len(protected)

CPU times: user 40.9 ms, sys: 321 ms, total: 362 ms
Wall time: 3.41 s


293

In [15]:
protectedGenes=set()
conjunto1=set()
conjunto2=set()
for i,r in enumerate(model2.reactions):
    if i in protected:
        conjunto1.update([g.id for g in r.genes])
    else:
        conjunto2.update([g.id for g in r.genes])
protectedGenes=conjunto1.difference(conjunto2)
len(protectedGenes)
    

209

In [16]:
def identificaTipo(nombre):
    if "(" in nombre:
        indice1=nombre.index("(")
        nombre2=nombre[indice1+1:]
        indice2=nombre2.index(")")
        nombre3=nombre2[:indice2] 
        if " and " in nombre3:
            tipo="SOP"
        if " or " in nombre3:
            tipo="POS"
    else:
        tipo="SOP"
    return(tipo)

def toPOS(nombre,tope=3):

    productos=set()    
    tipo=identificaTipo(nombre)

    if tipo=="SOP":
        sumandos=nombre.split(" or ")
        for sumando in sumandos:
            if " and " in sumando:
                casos=sumando.split(" and ")
                miCaso=set()
                for caso in casos:
                    caso2=caso.replace("(","")
                    caso3=caso2.replace(")","")
                    caso4=caso3.replace(" ","")
                    miCaso.add(caso4)

                productos.add(frozenset(miCaso))
            else:
                productos.add(frozenset([sumando]))

        CS=set([frozenset()])
        for producto in productos:
            CS2=set()
            for C in CS:
                if len(set(C).intersection(producto))==0:
                    for a in producto:
                        C2=set(C.copy())
                        C2.add(a)
                        if len(C2)<=tope:
                            CS2.add(frozenset(C2))
                else:
                    CS2.add(C)
            CS=CS2

        CS3=set()
        for C in CS:
            bien=True
            for C2 in CS:
                if not C==C2 and C2.issubset(C):
                    bien=False
                    break
            if bien:
                CS3.add(C)
    else:
        CS2=set()
        productos = nombre.split(" and ")
        for producto in productos:
            CS=set()
            componentes=producto.split(" or ")
            for C in componentes:
                C2=C.replace("(","")
                C3=C2.replace(")","")
                C4=C3.replace(" ","")
                CS.add(C4)
            CS2.add(frozenset(CS))
        CS3=set()
        for C in CS2:
            bien=True
            for C2 in CS2:
                if not C==C2 and C2.issubset(C):
                    bien=False
                    break
            if bien:
                CS3.add(C)
    return(CS3)

def asociar(modelo,tope=4):
    verbose=False
    total=0
    asociados=dict()
    reacciones=dict()

    for num in range(len(modelo.reactions)):
        nombre=modelo.reactions[num].gene_reaction_rule
        CS3=toPOS(nombre,tope)
        k=0
        while k<=tope:
            mas=0
            nivel=set([C for C in CS3 if len(C)==k and not C==frozenset({''})])
            if len(nivel)>0:
                mios=set()
                for caso in nivel:
                    mio=set()
                    for a in caso:
                        mio.add(a)
                    mios.add(frozenset(mio))

                asociados[(modelo.reactions[num],k)]=mios

                for caso in mios:
                    if not caso in reacciones:
                        reacciones[caso]=set([modelo.reactions[num]])
                    else:
                        reacciones[caso].add(modelo.reactions[num])
                mas+=len(nivel)
            total+=mas
            k+=1    
        if verbose:
            print(num,len(modelo.reactions[num].genes),mas)

    return(asociados,reacciones)
    
    
def soporteG(soporte,tope,modelo=model2):
    sopG=set()
    for j in soporte:
        r=modelo.reactions[j]
        for k in range(1,tope+1):
            if (r,k) in asociados:
                sopG.update(asociados[(r,k)])

    sopG1=set([frozenset(C) for C in sopG if len(protectedGenes.intersection(C))==0])
        
    sopG2=set()
    for caso in sopG1:
        poner=True
        for caso2 in sopG1:
            if not caso==caso2 and caso2.issubset(caso) and len(caso2)>0:
                poner=False
                break
        if poner:
            sopG2.add(caso)
    return(sopG2)

def chequea(inicio,fin):
    respuestas=set()
    for i in range(inicio,fin):
        if i<len(finales):
            C=finales[i]
            vale=True
            
            if C in powers:
                po=powers[C]
            else:
                po=set([frozenset(a) for a in mi.powerset(C) if len(a)>0])
                powers[C]=po

            if not powers[C].isdisjoint(todosMCS):
                vale=False

            if vale:
                for sop in listaSoportes[indice[C]:]:
                    if powers[C].isdisjoint(sopG[sop]):           
                        vale=False
                        break
            if vale:
                respuestas.add(C)

    return(respuestas)


In [17]:
def procesaTemporales(inicio,fin):
    miLista2=set()
    misFinales=set()
    for i in range(inicio,fin):
        if i<len(listaTemporales):
            caso=listaTemporales[i]
            for caso2 in sG:
                nuevo=caso.union(caso2)
                if len(nuevo)<=iMio:
                    if len(caso.union(caso2))<iMio:
                        miLista2.add(nuevo)
                    else:
                        misFinales.add(nuevo)
    return([miLista2,misFinales])

In [18]:
def CSV2(modelo2,tope,misReacciones,misValores,verbose,evita):
    global tiempos
    global todosMCS
    global evitar
    global misSoportes
    global listaMias
    global listaFinales
    global sopG, powers
    global modelo
    
    global miLista2
    global listaL2
    
    global lista3
    
    global finales
    global CSs1
    global salto
    global nuevos
    
    global listaSoportes, indice, iMio
    
    modelo=modelo2
    
    listaMias=[]
    saltos=[4]
    
    
    evitar=evita
    listaMCS=[]
    listaMCS.append(set())
    
    listaCandidatos=[]
    listaCandidatos.append(set([frozenset()]))
    todosMCS=set()
    cutsets=[set()]
    candidatos=[set()]

    ts=0
    
    t0s=0
    t1s=0
    t2s=0
    
    for i in range(1,tope+1):
        if i==1:      
            st=time.time()
            cpus=mp.cpu_count()
            cpus=64
            miL2=set()
            listaGenes=list(modelo.genes)

            k=0
            argus=[]
            num=int(np.floor(len(modelo.genes)/cpus))
            while k*num<len(listaGenes):
                argus.append((k*num,(k+1)*num))
                k+=1

            with mp.Pool(processes=cpus) as mp_pool:
                results=mp_pool.starmap(analizaGenes,argus)

            CS0=set()
            
            for result in results:
                CS0.update(result[0])
                misSoportes.update(result[1])
                
            print(len(CS0),len(misSoportes))
            todosMCS.update(CS0)    
            cutsets.append(CS0)
            
            for sop in misSoportes:
                sopG[sop]=soporteG(sop,tope,modelo)
            
            print("Final Fase 1 con",len(cutsets[1]),"cutsets")

            CSs1=set([g for caso in CS0 for g in caso])
            
            protectedGenes.update(CSs1)
            
            for sop in sopG:
                sopG[sop]=reduce(sopG[sop])
            print(time.time()-st)
            print()
       
        ############################## Final Fase 1 ############################################
    
        else:
            print("Inicio de Fase",i,"con",len(misSoportes),"soportes")
            iMio=i
            
            ############################## Parte 1.- Berge #################################### 
            st1=time.time()
            CST2=set()
            lista=set([frozenset()])
            descartados=set()
            cont2=0
            long=dict()
            indice=dict()
            analizados=set()
            
            for sop in misSoportes:
                long[sop]=len(sopG[sop])
            sortedSupports= sorted(long.items(), key=lambda x:x[1])
            finales=set()
            listaSoportes=[sortedSupports[t][0] for t in range(len(misSoportes))]
            
            for t in range(len(misSoportes)):
                ################ Para cada soporte #################
                item=sortedSupports[t]
                sop=item[0]
                
                lista2=set()
                temporales=set()
                
                tt=time.time()
                
                for caso in lista.difference(powers):
                    po=set([frozenset(a) for a in mi.powerset(caso) if len(a)>0])
                    powers[caso]=po
                
                t0s+=(time.time()-tt)   
                tt=time.time()
                for caso in lista:
                    if not powers[caso].isdisjoint(sopG[sop]):
                        lista2.add(caso)
                    else:
                        temporales.add(caso)
                t1s+=(time.time()-tt)   
                
                tt=time.time()
                for caso in temporales:
                    lista3=set([caso.union(caso2) for caso2 in sopG[sop] if len(caso.union(caso2))<iMio])
                    lista2.update(lista3)
                    lista3=set([caso.union(caso2) for caso2 in sopG[sop] if len(caso.union(caso2))==iMio])
                    finales.update(lista3)
                    
                    for C in lista3:
                        indice[C]=t
                    
                t2s+=(time.time()-tt)
                
                lista4=set()
                for C in lista2:
                    sst=time.time()
                    if C in powers:
                        po2=powers[C]
                    else:
                        po2=set([frozenset(a) for a in mi.powerset(C) if len(a)>0])
                        powers[C]=po2
                    if len(po2.intersection(todosMCS))==0 and len(evitar.intersection(C))==0:
                        lista4.add(C)
                    ts+=(time.time()-sst)
                cont2+=1
                lista=lista4
                lista3=[C for C in lista if len(C)<i]
                if verbose:
                    print("Mal",len(lista3))
            
                
            print("Finales",len(finales))
                    #break
            ########################## He terminado de procesar los soportes ###############
            
            if i==tope:
                print("Fase1",time.time()-st1)
            st2=time.time()
            
            if verbose:
                print("Parte paralela")
            
            lista3=[C for C in lista if len(C)<i]
            
            
            
            misFinales=list(finales)
            if verbose:
                print("Total",len(misFinales))
            lista=set()
            numLista=8*10**6
            r=0
            while r*numLista<len(misFinales):
                maximo=(r+1)*numLista
                if len(misFinales)<maximo:
                    maximo=len(misFinales)
                finales=misFinales[r*numLista:maximo]
                if verbose:
                    print(r,"Finales",r*numLista,maximo)
                r+=1
                

                finales=list(finales)


                cpus=mp.cpu_count()
                cpus=64
                k=0
                argus=[]
                num=int(np.floor(len(finales)/cpus))
                while k*num<len(finales):
                    argus.append((k*num,(k+1)*num))
                    k+=1

                with mp.Pool(processes=cpus) as mp_pool:
                    results=mp_pool.starmap(chequea,argus)

                for result in results:
                    lista.update(result)
                



            if i==tope:
                print("Fase2",time.time()-st2)
            st3=time.time()
            
            cont2=1
            
            listaL2=list(lista)
            
            for salto in saltos:
                nuevos=set()
                lista2=set()
                cpus=mp.cpu_count()
                cpus=64

                k=0
                argus=[]
                num=int(np.floor(len(listaL2)/cpus))
                while k*num<len(listaL2):
                    argus.append((k*num,(k+1)*num))
                    k+=1

                with mp.Pool(processes=cpus) as mp_pool:
                    results=mp_pool.starmap(analizaSalto,argus)

                for result in results:
                    lista2.update(result[0])
                    misSoportes.update(result[1])
                    nuevos.update(result[1])
                    for sop in result[1]:
                        if not sop in sopG:
                            sopG[sop]=soporteG(sop,tope,modelo)
                    cont2+=1
                
                
                
                #Refiltrar
                listaL2=list(lista2)
                for C in lista2:
                    if not C in powers:
                        powers[C]=set([frozenset(a) for a in mi.powerset(C) if len(a)>0])
                
                lista3=set()
                k=0
                argus=[]
                num=int(np.floor(len(listaL2)/cpus))
                while k*num<len(listaL2):
                    argus.append((k*num,(k+1)*num))
                    k+=1

                with mp.Pool(processes=cpus) as mp_pool:
                    results=mp_pool.starmap(filtra,argus)

                for result in results:
                    lista3.update(result)
                
                lista2=lista3
                listaL2=list(lista2)

                
                if verbose:
                    print()
                    print("Lista2",len(lista2))
                    print()
            
            cutsetsT=set()
            
            if i==tope:
                print("Fase3",time.time()-st3)
                miLista2=lista2
            st4=time.time()
            
            if verbose:
                print("Sin salto")
            descartados=set()
            cont=0
            if len(lista2)<100:
                cpus=2
            elif len(lista2)<1000:
                cpus=10
            else:
                cpus=64
            miLista2=list(lista2)
            listaT=set()

            k=0
            argus=[]
            num=int(np.floor(len(miLista2)/cpus))
            while k*num<len(miLista2):
                argus.append((k*num,(k+1)*num))
                k+=1

            with mp.Pool(processes=cpus) as mp_pool:
                results=mp_pool.starmap(analiza,argus)

            for result in results:
                listaT.update(result[0])
                misSoportes.update(result[1])
                for sop in result[1]:
                    if not sop in sopG:
                        sopG[sop]=soporteG(sop,tope,modelo)
            
            cutsets.append(set(listaT))
            if i==tope:
                print("Fase4",time.time()-st4)
            todosMCS.update(listaT)   
        if i>1:
            tiempos.append([st1,st2,st3])
    sopG=dict()
    powers=dict()
    misSoportes=set()
    
    print("Tiempos",t0s,t1s,t2s)
    return(cutsets)
                        

In [19]:
def reduce(sG):
    sopG1=set([frozenset(C) for C in sG if len(protectedGenes.intersection(C))==0])
    sopG2=set()
    for caso in sopG1:
        poner=True
        for caso2 in sopG1:
            if not caso==caso2 and caso2.issubset(caso) and len(caso2)>0:
                poner=False
                break
        if poner:
            sopG2.add(caso)
    return(sopG2)

In [20]:
def analizaSalto(inicio,fin):
    misSoportes2=set()
    
    descartados=set()
    miLista2=set()
    
    i2=inicio
    while i2 < fin:
        casos=[]
        contCasos=0
        for k in range(i2,fin):
            if k<len(listaL2) and not listaL2[k] in descartados:
                casos.append(listaL2[k])
                contCasos+=1
            if contCasos>=salto:
                break
        i2=k+1

        cont3=0
        if len(casos) > 0:
            with modelo:
                for caso in casos:
                    descartados.add(caso)
                    for g in caso:
                        modelo.genes.get_by_id(g).knock_out()
                for i in range(len(misReacciones)):
                    modelo.reactions[misReacciones[i]].bounds=misValores[i]
                
                sol=modelo.slim_optimize(error_value=-1)
                if sol==-1:
                    for caso in casos:
                        miLista2.add(caso)
                else:
                    sol2=modelo.optimize()
                    sop=frozenset([j for j in range(len(modelo.reactions)) if abs(sol2.fluxes[j])>10**-8 and not j in protected])
                    if not sop in sopG:
                        sopG[sop]=soporteG(sop,tope,modelo)
                    misSoportes2.add(sop)
                    for C in set(listaL2).difference(descartados): 
                        sst=time.time()
                        if C in powers:
                            sub=powers[C]
                        else:
                            sub=set([frozenset(a) for a in mi.powerset(C) if len(a)>0])
                            powers[C]=sub
                        if len(sub.intersection(sopG[sop]))==0:
                            descartados.add(C)
                        

    return([miLista2,misSoportes2])

In [21]:
def analiza(inicio,fin):    
    misSoportes2=set()
    descartados=set()
    CSs=set()
    for caso in miLista2[inicio:fin]:
        if not caso in descartados:
            descartados.add(caso)
            with modelo:
                for g in caso:
                    modelo.genes.get_by_id(g).knock_out()
                for i in range(len(misReacciones)):
                    modelo.reactions[misReacciones[i]].bounds=misValores[i]
                sol=modelo.slim_optimize(error_value=-1)
                if sol==-1:
                    CSs.add(caso)
                else:
                    sol2=modelo.optimize()
                    sop=frozenset([j for j in range(len(modelo.reactions)) if abs(sol2.fluxes[j])>10**-8 and not j in protected])
                    if not sop in sopG:
                        sopG[sop]=soporteG(sop,tope,modelo)
                    misSoportes2.add(sop)
                    for C in set(miLista2).difference(descartados): 
                        sst=time.time()
                        sub=set([frozenset(a) for a in mi.powerset(C) if len(a)>0])
                        if len(sub.intersection(sopG[sop]))==0:
                            descartados.add(C)
                        
    return([CSs,misSoportes2])

In [22]:
def analizaGenes(inicio,fin):
    correctas=set()
    soportes=set()
    for i in range(inicio,fin):
        if i<len(listaGenes):
            g=modelo.genes.get_by_id(listaGenes[i].id)
            with modelo:
                if not g.id in protectedGenes:
                    g.knock_out()
                    for i in range(len(misReacciones)):
                        modelo.reactions[misReacciones[i]].bounds=misValores[i]
                    sol=modelo.slim_optimize(error_value=-1)
                    if sol==-1:
                        correctas.add(frozenset([g.id]))
                    else:
                        sol2=modelo.optimize(objective_sense="maximize")
                        sop=frozenset([j for j in range(len(modelo.reactions)) if abs(sol2.fluxes[j])>10**-8 and not j in protected])
                        soportes.add(sop)
    return([correctas,soportes])

In [23]:
def filtra(inicio,fin):    
    CSs=set()
    for caso in listaL2[inicio:fin]:
        vale=True
        for sop in nuevos:
            po=powers[caso]
            sG=sopG[sop]
            if len(po.intersection(sG))==0:
                vale=False
                break
        if vale:
            CSs.add(caso)
                        
    return(CSs)

In [24]:
misReacciones, misValores = [biomass,etoh],[[0.1,1000],[0,0]]

In [28]:
%%time
modelo=model2.copy()
modelo.reactions[o2].bounds=[0,0]
listaGenes=list(modelo.genes)
misSoportes=set()
tope=4
asociados,reacciones=asociar(modelo,tope)

for i in protected:
    r=modelo.reactions[i]
    if (r,1) in asociados:
        for C in asociados[(r,1)]:
            for g in C:
                protectedGenes.add(g)
protectedGenes2=set()
prots2=set()
'''for i in protected:
    r=modelo.reactions[i]
    if (r,2) in asociados:
        protectedGenes2.update(asociados[(r,2)])
        for C in asociados[(r,2)]:
            prots2.update(C)
'''
print("Protected",len(protectedGenes))
sopG=dict()
tiempos=[]
powers=dict()
CS1=CSV2(modelo,tope,misReacciones,misValores,True,set())

Protected 255
0 116
Final Fase 1 con 0 cutsets
2.690465211868286

Inicio de Fase 2 con 116 soportes
Mal 50
Mal 49
Mal 40
Mal 38
Mal 37
Mal 37
Mal 36
Mal 36
Mal 35
Mal 35
Mal 35
Mal 34
Mal 34
Mal 34
Mal 34
Mal 34
Mal 33
Mal 33
Mal 22
Mal 22
Mal 20
Mal 19
Mal 17
Mal 17
Mal 17
Mal 17
Mal 17
Mal 15
Mal 15
Mal 15
Mal 15
Mal 15
Mal 15
Mal 15
Mal 15
Mal 15
Mal 15
Mal 15
Mal 15
Mal 15
Mal 15
Mal 15
Mal 15
Mal 15
Mal 15
Mal 14
Mal 14
Mal 14
Mal 14
Mal 14
Mal 14
Mal 14
Mal 14
Mal 14
Mal 14
Mal 14
Mal 14
Mal 14
Mal 14
Mal 14
Mal 14
Mal 14
Mal 14
Mal 14
Mal 14
Mal 14
Mal 14
Mal 13
Mal 13
Mal 13
Mal 13
Mal 13
Mal 13
Mal 13
Mal 13
Mal 13
Mal 13
Mal 13
Mal 13
Mal 13
Mal 13
Mal 13
Mal 13
Mal 12
Mal 12
Mal 11
Mal 11
Mal 11
Mal 11
Mal 10
Mal 10
Mal 10
Mal 10
Mal 9
Mal 9
Mal 7
Mal 7
Mal 6
Mal 6
Mal 5
Mal 4
Mal 4
Mal 4
Mal 3
Mal 3
Mal 2
Mal 2
Mal 2
Mal 2
Mal 2
Mal 2
Mal 2
Mal 1
Mal 0
Mal 0
Mal 0
Finales 1960
Parte paralela
Total 1960
0 Finales 0 1960

Lista2 99

Sin salto
Inicio de Fase 3 con 300 soportes

Mal 644
Mal 643
Mal 633
Mal 628
Mal 628
Mal 622
Mal 621
Mal 620
Mal 619
Mal 619
Mal 617
Mal 616
Mal 615
Mal 612
Mal 611
Mal 611
Mal 603
Mal 563
Mal 561
Mal 561
Mal 561
Mal 555
Mal 555
Mal 555
Mal 555
Mal 555
Mal 555
Mal 555
Mal 554
Mal 550
Mal 545
Mal 534
Mal 530
Mal 528
Mal 528
Mal 492
Mal 492
Mal 492
Mal 491
Mal 487
Mal 485
Mal 484
Mal 483
Mal 481
Mal 475
Mal 475
Mal 475
Mal 474
Mal 469
Mal 461
Mal 461
Mal 459
Mal 427
Mal 425
Mal 422
Mal 421
Mal 421
Mal 394
Mal 394
Mal 394
Mal 394
Mal 382
Mal 382
Mal 382
Mal 382
Mal 382
Mal 382
Mal 382
Mal 380
Mal 380
Mal 378
Mal 376
Mal 376
Mal 376
Mal 376
Mal 376
Mal 375
Mal 373
Mal 365
Mal 353
Mal 352
Mal 352
Mal 352
Mal 351
Mal 350
Mal 348
Mal 348
Mal 342
Mal 341
Mal 341
Mal 320
Mal 319
Mal 318
Mal 316
Mal 294
Mal 294
Mal 293
Mal 292
Mal 290
Mal 289
Mal 280
Mal 267
Mal 265
Mal 265
Mal 263
Mal 257
Mal 247
Mal 242
Mal 242
Mal 239
Mal 238
Mal 236
Mal 232
Mal 228
Mal 192
Mal 192
Mal 161
Mal 161
Mal 159
Mal 126
Mal 120
Mal 118
Mal 118
Mal 118
Mal 112


In [29]:
total=0
for i,C in enumerate(CS1):
    print(i,len(C))
    total+=len(C)
print(total)

0 0
1 0
2 81
3 223
4 509
813
